<a href="https://colab.research.google.com/github/w95wayne10/personal_financial_management/blob/master/DH_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0:  
0:  
[ index:(0, 0), 
$h^2$*1e5 = 10.96, 
$\mu$*1e5 = -5.48, 
$\eta$ = 3, 
$p_u$ = 0.3682, 
$p_m$ = 0.2591, 
$p_d$ = 0.3727, 
price = 0.7225, 
$\Delta$ = 0.50397 ,  
index:(0, 0), 
$h^2$*1e5 = 10.96, 
$\mu$*1e5 = -5.48, 
$\eta$ = 3, 
$p_u$ = 0.3682, 
$p_m$ = 0.2591, 
$p_d$ = 0.3727, 
price=0.7225, 
$\Delta$ = 0.50397 ]  

1:  
-3:  
[ index:(1, -3), 
$h^2$*1e5 = 11.1079, 
$\mu$*1e5 = -5.554, 
$\eta$ = 3, 
$p_u$ = 0.3732, 
$p_m$ = 0.2491, 
$p_d$ = 0.3777, 
price=0.1726, 
$\Delta$ = 0.19247 ,  
index:(1, -3), 
$h^2$*1e5 = 11.1079, 
$\mu$*1e5 = -5.554, 
$\eta$ = 3, 
$p_u$ = 0.3732, 
$p_m$ = 0.2491, 
$p_d$ = 0.3777, 
price=0.1726, 
$\Delta$ = 0.19247 ]  
0:  
[ index:(1, 0), 
$h^2$*1e5 = 10.5215, 
$\mu$*1e5 = -5.2608, 
$\eta$ = 3, 
$p_u$ = 0.3535, 
$p_m$ = 0.2888, 
$p_d$ = 0.3578, 
price=0.55274, 
$\Delta$ = 0.50304 ,  
index:(1, 0), 
$h^2$*1e5 = 10.5215, 
$\mu$*1e5 = -5.2608, 
$\eta$ = 3, 
$p_u$ = 0.3535, 
$p_m$ = 0.2888, 
$p_d$ = 0.3578, 
price=0.55274, 
$\Delta$ = 0.50304 ]  
3:  
[ index:(1, 3), 
$h^2$*1e5 = 11.1186, 
$\mu$*1e5 = -5.5593, 
$\eta$ = 3, 
$p_u$ = 0.3735, 
$p_m$ = 0.2484, 
$p_d$ = 0.3781, 
price=1.39859, 
$\Delta$ = 0.81218 ,  
index:(1, 3), 
$h^2$*1e5 = 11.1186, 
$\mu$*1e5 = -5.5593, 
$\eta$ = 3, 
$p_u$ = 0.3735, 
$p_m$ = 0.2484, 
$p_d$ = 0.3781, 
price=1.39859, 
$\Delta$ = 0.81218 ]  

2:  
-6:  
[ index:(2, -6), 
$h^2$*1e5 =  = 11.241, 
$\mu$*1e5 = -5.6205, 
$\eta$ = 3, 
$p_u$ = 0.3776, 
$p_m$ = 0.2401, 
$p_d$ = 0.3822, 
price=0.0, 
$\Delta$ = 0.0 ,  
index:(2, -6), 
$h^2$*1e5 = 11.241, 
$\mu$*1e5 = -5.6205, 
$\eta$ = 3, 
$p_u$ = 0.3776, 
$p_m$ = 0.2401, 
$p_d$ = 0.3822, 
price=0.0, 
$\Delta$ = 0.0 ]  
-3:  
[ index:(2, -3), 
$h^2$*1e5 = 10.6546, 
$\mu$*1e5 = -5.3273, 
$\eta$ = 3, 
$p_u$ = 0.3579, 
$p_m$ = 0.2798, 
$p_d$ = 0.3623, 
price=0.0, 
$\Delta$ = 0.0 ,  
index:(2, -3), 
$h^2$*1e5 = 10.7135, 
$\mu$*1e5 = -5.3567, 
$\eta$ = 3, 
$p_u$ = 0.3599, 
$p_m$ = 0.2758, 
$p_d$ = 0.3643, 
price=0.0, 
$\Delta$ = 0.0 ]  
0:  
[ index:(2, 0), 
$h^2$*1e5 = 10.1269, 
$\mu$*1e5 = -5.0634, 
$\eta$ = 3, 
$p_u$ = 0.3402, 
$p_m$ = 0.3154, 
$p_d$ = 0.3444, 
price=0.41631, 
$\Delta$ = 0.50304 ,  
index:(2, 0), 
$h^2$*1e5 = 11.2518, 
$\mu$*1e5 = -5.6259, 
$\eta$ = 3, 
$p_u$ = 0.378, 
$p_m$ = 0.2394, 
$p_d$ = 0.3826, 
price=0.46255, 
$\Delta$ = 0.50304 ]  
3:  
[ index:(2, 3), 
$h^2$*1e5 = 10.6642, 
$\mu$*1e5 = -5.3321, 
$\eta$ = 3, 
$p_u$ = 0.3582, 
$p_m$ = 0.2791, 
$p_d$ = 0.3626, 
price=1.22372, 
$\Delta$ = 1.0 ,  
index:(2, 3), 
$h^2$*1e5 = 10.7237, 
$\mu$*1e5 = -5.3619, 
$\eta$ = 3, 
$p_u$ = 0.3602, 
$p_m$ = 0.2751, 
$p_d$ = 0.3647, 
price=1.22372, 
$\Delta$ = 1.0 ]  
6:  
[ index:(2, 6), 
$h^2$*1e5 = 11.2614, 
$\mu$*1e5 = -5.6307, 
$\eta$ = 3, 
$p_u$ = 0.3783, 
$p_m$ = 0.2388, 
$p_d$ = 0.3829, 
price=2.46242, 
$\Delta$ = 1.0 ,  
index:(2, 6), 
$h^2$*1e5 = 11.2614, 
$\mu$*1e5 = -5.6307, 
$\eta$ = 3, 
$p_u$ = 0.3783, 
$p_m$ = 0.2388, 
$p_d$ = 0.3829, 
price=2.46242, 
$\Delta$ = 1.0 ]  

3:  
-9:[index:(3, -9), 
$h^2$*1e5 = 11.3607, 
$\mu$*1e5 = -5.6803, 
$\eta$ = 3, 
$p_u$ = 0.3816, 
$p_m$ = 0.232, 
$p_d$ = 0.3863, 
price=0, 
$\Delta$ = inf]  
-6:[index:(3, -6), 
$h^2$*1e5 = 10.7744, 
$\mu$*1e5 = -5.3872, 
$\eta$ = 3, 
$p_u$ = 0.3619, 
$p_m$ = 0.2717, 
$p_d$ = 0.3664, 
price=0, 
$\Delta$ = inf]  
-3:[index:(3, -3), 
$h^2$*1e5 = 10.2467, 
$\mu$*1e5 = -5.1233, 
$\eta$ = 3, 
$p_u$ = 0.3442, 
$p_m$ = 0.3073, 
$p_d$ = 0.3484, 
price=0, 
$\Delta$ = inf]  
0:[index:(3, 0), 
$h^2$*1e5 = 9.7717, 
$\mu$*1e5 = -4.8858, 
$\eta$ = 3, 
$p_u$ = 0.3283, 
$p_m$ = 0.3395, 
$p_d$ = 0.3323, 
price=0.0, 
$\Delta$ = inf]  
3:[index:(3, 3), 
$h^2$*1e5 = 10.2553, 
$\mu$*1e5 = -5.1277, 
$\eta$ = 3, 
$p_u$ = 0.3445, 
$p_m$ = 0.3068, 
$p_d$ = 0.3487, 
price=1.22372, 
$\Delta$ = inf]  
6:[index:(3, 6), 
$h^2$*1e5 = 10.7928, 
$\mu$*1e5 = -5.3964, 
$\eta$ = 3, 
$p_u$ = 0.3626, 
$p_m$ = 0.2704, 
$p_d$ = 0.367, 
price=2.46242, 
$\Delta$ = inf]  
9:[index:(3, 9), 
$h^2$*1e5 = 11.39, 
$\mu$*1e5 = -5.695, 
$\eta$ = 3, 
$p_u$ = 0.3826, 
$p_m$ = 0.2301, 
$p_d$ = 0.3873, 
price=3.71628, 
$\Delta$ = inf]  

A hedger is short 10,000 European calls.  
As an option covers 100 shares of stock, N = 1,000,000.  


0 3 6 3

In [1]:
import numpy as np

In [ ]:
S = 100
node_index = np.array([0,3,6,3])
y = np.log(S)+node_index*np.sqrt(0.00006575)/2
S = np.exp(y)
S

array([100.        , 101.22372256, 102.46242009, 101.22372256])

In [ ]:
f = np.array([0.7225, 1.39859, 2.46242, 1.22372])

In [ ]:
delta = np.array([0.50397, 0.81218, 1.0, 1.0])
delta

array([0.50397, 0.81218, 1.     , 1.     ])

In [ ]:
N = 1000000

In [ ]:
change_delta = delta.copy()
change_delta[1:] = delta[1:]-delta[:-1]
change_delta

array([0.50397, 0.30821, 0.18782, 0.     ])

In [ ]:
shares_bought = N*change_delta
shares_bought

array([503970., 308210., 187820.,      0.])

In [ ]:
cost_shares = S*shares_bought
cost_shares

array([50397000.00000002, 31198163.53059283, 19244491.74158613,
              0.        ])

In [ ]:
cumulative_cost = np.cumsum(cost_shares)
cumulative_cost

array([5.03970000e+07, 8.15951635e+07, 1.00839655e+08, 1.00839655e+08])

In [ ]:
cumulative_cost[-1] - N*S[0]

839655.2721789479

Monte Carlo

In [19]:
#yt1 = yt0 + r - ht**2/2 + ht*epsilon
S = 100
y0 = np.log(S)
yt = y0
h=np.sqrt(10.96/1e5)
np.random.seed(0)
print(np.exp(yt), h**2*1e5)
price_path = []
price_path.append(S)
for t in range(3):
  eps = np.random.standard_normal(1)[0]
  yt = yt - h**2/2 + h*eps
  h = np.sqrt(0.000006575 + 0.9*h**2 + 0.04*h**2*eps**2)
  price_path.append(np.exp(yt))
  print(np.exp(yt), h**2*1e5, eps)
price_path

100.00000000000004 10.959999999999999
101.8583633090849 11.885748489811265 1.764052345967664
102.29762019136398 11.430802236166556 0.4001572083672233
103.36779218047606 11.383217452518839 0.9787379841057392


[100, 101.8583633090849, 102.29762019136398, 103.36779218047606]

In [22]:
price_range = np.exp(np.log(100)+(np.arange(9)-4)*3*np.sqrt(0.00006575)/2)
price_range

array([ 95.25127138,  96.41688268,  97.59675783,  98.79107137,
       100.        , 101.22372256, 102.46242009, 103.71627584,
       104.98547531])

In [21]:
for p in price_path:
  print(np.argmin(np.abs(p - price_range)))

4
6
6
7
